In [124]:
from astropy.io import fits
from astropy.time import Time
from astropy.wcs import WCS
from astroquery.jplhorizons import Horizons
from _mypath import thisdir
import numpy as np
import os

In [125]:
def get_path(rela_path):
    """link the relative path to the path of this dir
    to get an absolute path
    
    Inputs: relative path, string
    Outputs: absolute path, string
    """
    return os.path.join(thisdir, rela_path)

In [128]:
def make_obs_log(horizon_id,
                 input_path, 
                 map_type, 
                 output_path):
    """creat an observing log file before all work,
    to provide non-data info for every extension.
    
    Inputs: 1. Horizon ID of the object, string;
            2. path of the folder containing dirs of every observation, string;
            3. 'sk' or 'rw' or 'ex', expected to be 'sk', string;
            4. ouput path of the newly generated obs log, string
            
    Outputs: No, only generate an obs log file
    """
    # initiate quantity names. w: write; r: read; v: value
    # make the code easier to be modified or appended
    obs     = {"w": "OBS_ID"}
    ext     = {"w": "EXTENSION"}
    start_t = {"w": "START",
               "r": "DATE-OBS"}
    end_t   = {"w": "END",
               "r": "DATE-END"}
    mid_t   = {"w": "MIDTIME"}
    exp     = {"w": "EXP_TIME",
               "r": "EXPOSURE"}
    fil     = {"w": "FILTER",
               "r": "FILTER"}
    pa      = {"w": "PA",
               "r": "PA_PNT"}    
    helio   = {"w": "HELIO",
               "r": "r"}
    helio_v = {"w": "HELIO_V",
               "r": "r_rate"}
    phase   = {"w": "PHASE",
               "r": "alpha"}
    ra      = {"w": "RA",
               "r": "RA"}
    dec     = {"w": "DEC",
               "r": "DEC"}
    px      = {"w": "PX"}
    py      = {"w": "PY"}
    
    # initiate a log file
    f = open(output_path, 'w')
    f.write(  obs["w"]     + ' ' 
            + ext["w"]     + ' ' 
            + start_t["w"] + ' ' 
            + end_t["w"]   + ' ' 
            + mid_t["w"]   + ' ' 
            + exp["w"]     + ' ' 
            + fil["w"]     + ' '
            + pa["w"]      + ' ' 
            + helio["w"]   + ' '
            + helio_v["w"] + ' ' 
            + phase["w"]   + ' '
            + ra["w"]      + ' '
            + dec["w"]     + ' '
            + px["w"]      + ' '
            + py["w"]      + '\n'
            )
    
    # start a loop iterating over observations
    obs_list = os.listdir(input_path)
    obs_list.sort()
    for obs_id in obs_list:
        obs["v"] = obs_id
        # find the only *sk* file for every obs
        map_dir_path = os.path.join(input_path, 
                                    obs_id+'/uvot/image/')
        map_file_list = os.listdir(map_dir_path)
        map_file = [x for x in map_file_list 
                   if (map_type in x)]
        if len(map_file) == 1:
            map_file = map_file[0]
        else:
            break
        map_file_path = os.path.join(map_dir_path, 
                                     map_file)
        
        # get the num of extensions to
        # start a loop interating over extensions
        hdul = fits.open(map_file_path)
        ext_num = len(hdul) - 1
        for ext_id in range(1, 1 + ext_num):
            # read observing info from header of every extension
            # and put the info into the log file
            ext["v"]     = ext_id
            ext_header   = hdul[ext_id].header
            start_t["v"] = Time(ext_header[start_t["r"]])
            end_t["v"]   = Time(ext_header[end_t["r"]])
            exp["v"]     = ext_header[exp["r"]]
            dt           = end_t["v"] - start_t["v"]
            mid_t["v"]   = start_t["v"] + 1/2*dt
            fil["v"]     = ext_header[fil["r"]]
            pa["v"]      = ext_header[pa["r"]]
            f.write(  obs["v"]           + ' '
                    + f'{int(ext["v"])}' + ' '
                    + f'{start_t["v"]}'  + ' '
                    + f'{end_t["v"]}'    + ' '
                    + f'{mid_t["v"]}'    + ' '
                    + f'{exp["v"]}'      + ' '
                    + fil["v"]           + ' '
                    + f'{pa["v"]}'       + ' ')
            
            # read ephmerides info from Horizon API
            # and put the info into the log file
            obj = Horizons(id=horizon_id, 
                           location='@swift',
                           epochs=mid_t["v"].jd)
            eph = obj.ephemerides()[0]
            helio["v"] = eph[helio["r"]]
            helio_v["v"] = eph[helio_v["r"]]
            phase["v"] = eph[phase["r"]]
            ra["v"] = eph[ra["r"]]
            dec["v"] = eph[dec["r"]]
            f.write(  f'{helio["v"]}'    + ' '
                    + f'{helio_v["v"]}'  + ' '
                    + f'{phase["v"]}'    + ' '
                    + f'{ra["v"]}'       + ' '
                    + f'{dec["v"]}'      + ' ')
            w = WCS(ext_header)
            px["v"], py["v"] = w.wcs_world2pix( ra["v"], dec["v"], 1)
            f.write(  f'{px["v"]}'       + ' '
                    + f'{py["v"]}'       + '\n')
    f.close()
            

In [129]:
# Borisov observed by Swift UVOT in Sep 2019
horizon_id  = 90004423
input_rela  = '../data/Borisov_raw/'
input_path  = get_path(input_rela)
map_type    = 'sk'
output_rela = '../docs/obs-log_Borisov_ZX.txt'
output_path = get_path(output_rela)

make_obs_log(horizon_id,
             input_path,
             map_type,
             output_path)